# OctoPod - Podcast Analyzer

**Setup (first time):** Add secrets via 🔑 icon:
- `ANTHROPIC_API_KEY`
- `GCS_CREDENTIALS` (service account JSON)

**Profiles:** Select a profile below to analyze different podcast categories (e.g., draft-fpl, politics, finance).

In [ ]:
# Setup - Run once per session
import os
os.chdir('/content')
!rm -rf OctoPod
!git clone https://github.com/CoaseToCoase/OctoPod.git
%cd OctoPod
!pip install -q -e .
print('\n✓ Setup complete')

In [ ]:
# Load secrets
import os
from google.colab import userdata

os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
os.environ['GCS_CREDENTIALS'] = userdata.get('GCS_CREDENTIALS')
print('✓ Secrets loaded')

In [ ]:
# Select profile
from octopod.config import set_profile, get_profile, list_profiles

# Change this to use a different profile
PROFILE = 'draft-fpl'

# Available profiles
print('Available profiles:', list_profiles())
set_profile(PROFILE)
print(f'✓ Using profile: {get_profile()}')

In [ ]:
# Sync existing data from GCS
from octopod.gcs import sync_data_from_gcs, is_gcs_configured
from octopod.config import get_profile, get_gcs_config
from octopod.data import init_db

init_db()
if is_gcs_configured():
    if sync_data_from_gcs():
        gcs = get_gcs_config()
        print(f'✓ Data synced from GCS for profile: {get_profile()}')
        print(f'  gs://{gcs["bucket"]}/{gcs["path_prefix"]}/data/')
    else:
        print('⚠ No existing data in GCS (first run)')
else:
    print('⚠ GCS not configured - check secrets')

In [ ]:
# Run the pipeline
!octopod --profile $PROFILE run

In [ ]:
# Save all data to GCS
from octopod.gcs import sync_data_to_gcs
from octopod.config import get_profile, get_gcs_config

if sync_data_to_gcs():
    gcs = get_gcs_config()
    print(f'✓ Data saved to GCS for profile: {get_profile()}')
    print(f'  gs://{gcs["bucket"]}/{gcs["path_prefix"]}/data/videos.json')
    print(f'  gs://{gcs["bucket"]}/{gcs["path_prefix"]}/data/analyses.json')
    print(f'  gs://{gcs["bucket"]}/{gcs["path_prefix"]}/summaries/')
else:
    print('⚠ Failed to sync to GCS')